# TIAFI extract physio data

[TIAFI's](https://tiafi.org/) 'extract physio data' notebook extracts and translates physiotherapy patient data from Arabic written word documents for an NGO helping Syrian refugees in Izmir,Turkey. Information from 43 documents is used as the basis for a new Retool physiotherapy database. The cells below allow new data to be added iteratively as additional documents continue to be found on trainers' computers.

### Import libraries

In [1]:
# imports libraries
import win32com.client as win32
import translators as ts
import os
import warnings
import math
import pandas as pd
pd.options.mode.chained_assignment = None

# local folder path
folder_path = r'C:\\Users\\HUAWEI\\Desktop\\TIAFI\\Physiotherapy database\\Extract scores\\Arabic\\'

Using state Casablanca-Settat server backend.


## 1. Functions

### 1.1 Extract scores

These functions extract the scores data by calculating an average for each week for each type of activity (ranging from 1-9) attempted by the physiotherapy instructors (not all activities are attempted each week if the patient is struggling with less challenging movements).

In [2]:
# cleans the numbers from a table cell by 
# dropping unnecessary punctuation/syntax. 
def clean_score(score):
    try:
        score = float(score.split("\r")[0])
        if score >= 0:
            return score
    except:
        return float('NaN')

# gets the average for non-nan values in a list. 
def get_avg(data):
    
    # gets the total for non-nan values. 
    total = sum([x for x in data if math.isnan(x) == False])
    
    # gets the number to divide our total by. 
    division_num = 0
    for i in range(len(data)):
        if data[i] >= 0:
            division_num = division_num + 1
    
    # returns the week average. 
    if division_num > 0:
        return total / division_num
    else:
        return float('NaN')
    

# gets the average for a specifc week. 
def get_week_avg(table, col):
    week_scores = []
    for i in range(2,len(tables[table].rows)):
        score = tables[table].Cell(Row = i, Column = col).Range.Text
        week_scores.append(clean_score(score))
        
    return get_avg(week_scores)

# gets all week averages for a table. 
def get_all_week_avgs(tables, table):
    week_avgs = []
    for i in range(2, len(tables[table].columns)+1):
        week_avgs.append(get_week_avg(table, col=i))
        
    return week_avgs

# gets all table week averages. 
def get_all_table_avgs(tables):
    table_avgs = []
    for i in [1,2,3,5,6,7,8]:
        table_avgs.append(get_all_week_avgs(tables, table=i))
        
    return table_avgs

### 1.2 Create additional columns

These functions create additonal columns for names and conditional, help to remedy a table formatting error whereby activities 4 and 5 are stored as part of the same table, and calulates a weekly total across all activities (a total being more informative than an average because certain patients may attempt a greater number activities but score less well on the more difficult ones).

In [3]:
# creates a list of 25 instances of the relevant data point (for each week). 

# gets the arabic name. 
def get_name(tables):
    name = []
    contents = tables[0].Cell(Row = 1, Column = 2).Range.Text
    for i in range(1,25):
        name.append(contents.split("\r")[0])
    return name


# gets the condition.
def get_condition(tables):
    condition = []
    contents = tables[0].Cell(Row = 3, Column = 2).Range.Text
    for i in range(1,25):
        condition.append(contents.split("\r")[0])
    return condition


# gets the average for a specifc week. 
def get_table_4_scores(tables):
    
    table = tables[4]
    all_week_avgs = []
    for week in range(2,26):
    
        week_scores = []
        for i in range(2,11):
            score = table.Cell(Row = i, Column = week).Range.Text
            week_scores.append(clean_score(score))
            
        week_avg = get_avg(week_scores)
        
        all_week_avgs.append(week_avg)
        
    return all_week_avgs


# gets the average for a specifc week. 
def get_table_5_scores(tables):
    
    table = tables[4]
    all_week_avgs = []
    for week in range(2,26):
        
        week_scores = []
        for i in range(14,18):
            score = table.Cell(Row = i, Column = week).Range.Text
            week_scores.append(clean_score(score))
            
        week_avg = get_avg(week_scores)
        
        all_week_avgs.append(week_avg)
        
    return all_week_avgs


# gets the scores for a given month. 
def get_week_total(scores, week):
    week_scores = []
    for i in range(len(scores)):
        week_scores.append(scores[i][week])
    return week_scores

## 2. Update dataframe

### 2.1 Extract scores

This cell returns a dataframe containing information on each patient for each week. Activities and weeks where the no score related information has been gathered is lablled as NaN instead of 0 for the purpose of calculating a weekly total (see below) once new data (if done iteratively) is added.

In [4]:
# creates a data frame for a single document. 
def get_doc_data(path):
    
    # reads in the word document. 
    word = win32.Dispatch('Word.Application')
    word.Visible = False
    word.Documents.Open(path)
    doc = word.ActiveDocument
    
    # creates a tables oject containing each of 
    # the document's tables. 
    global tables
    tables = []
    for i in range(1, doc.Tables.Count):
        tables.append(doc.Tables(i))
    
    # creates a list of scores for each question. 
    scores = get_all_table_avgs(tables)
    
    # creates a data frame of relevant data points. 
    d = {'Name (ar)': get_name(tables),
         'Condition (ar)': get_condition(tables),
         'Week': list(range(1,25)),
         'Activity 1': scores[0],
         'Activity 2': scores[1],
         'Activity 3': scores[2],
         'Activity 4': get_table_4_scores(tables),
         'Activity 5': get_table_5_scores(tables),
         'Activity 6': scores[3],
         'Activity 7': scores[4],
         'Activity 8': scores[5],
         'Activity 9': scores[6]}
    
    results_df = pd.DataFrame(d)
    
    return results_df

# creates a list of data frames for multiple documents. 
def get_multiple_docs(folder_path):
    
    # creates a list of document names. 
    file_names = os.listdir(folder_path)
    
    # filters out any temporary documents. 
    file_names = [x for x in file_names if '~' not in x ]
    # filters out any spreadsheets. 
    file_names = [x for x in file_names if '.csv' not in x ]
    # filters out any jupyter notebooks. 
    file_names = [x for x in file_names if '.ipynb' not in x ]
    
    # extracts only the file name and drops all else. 
    for i in range(len(file_names)):
        file_names[i] = file_names[i].split('.docx')[0]
    
    # creates a list of scores for each document. 
    scores = []
    for i in range(len(file_names)):
        file_name = file_names[i]
        path = folder_path + file_name + '.docx'
        scores.append(get_doc_data(path))
    
    return scores

# stores the scores from multiple documents in a table. 
scores = get_multiple_docs(folder_path)
final_df = pd.concat(scores[0:len(scores)]).reset_index(drop=True)
final_df.head(4)

,Name (ar),Condition (ar),Week,Activity 1,Activity 2,Activity 3,Activity 4,Activity 5,Activity 6,Activity 7,Activity 8,Activity 9
0,احمد الحسين,شلل دماغي,1,1.666667,1.888889,0.315789,NaN,NaN,NaN,NaN,NaN,NaN
1,احمد الحسين,شلل دماغي,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,احمد الحسين,شلل دماغي,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,احمد الحسين,شلل دماغي,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.2 Get new rows

The new dataframe is compared with the old dataframe to check which rows are new and store them as a seperate dataframe. This is useful for transaltion (see below) in case an initial translation does not work due to the Google API rate limit being reached. 

In [6]:
def get_new_rows(folder_path, final_df):
    
    # reads in the old data frame. 
    global old_df
    old_df = pd.read_csv(folder_path + 'Physiotherapy Database.csv')

    new_rows = []
    # only runs if there is already data present in the spreadsheet. 
    # otherwise it returns the final_df. 
    if len(old_df) > 0:
        
        # gets a list of existing names and months. 
        existing_details = []
        for i in range(len(old_df)):
            existing_details.append(list(old_df.iloc[i,[0, 4]]))
        
        # loops over the data frame to find new rows. 
        for i in range(len(final_df)):
            
            final_details = [final_df.iloc[i,[0, 2]][0], final_df.iloc[i,[0, 2]][1]]
            if not final_details in existing_details:
                
                # checks whether the name and condition are present. 
                if len(final_df.iloc[i,0]) > 0 and len(final_df.iloc[i,1]) > 0:
                    
                    # appends new rows
                    new_rows.append(i)
                    
                else:
                    # sends a warning message about a missing name. 
                    if len(final_df.iloc[i,0]) > 0:
                        message = 'Document ' + str(i+1) + ' has a missing name. Please fill it in.'
                        warnings.warn(message)
                    
                    # sends a warning message about a missing condition. 
                    if len(final_df.iloc[i,1]) > 0:
                        message = 'Document ' + str(i+1) + ' has a missing condition. Please fill it in.'
                        warnings.warn(message)

    # if there is no data in the spreadsheet the final_df is returned. 
    else: 
        new_rows = range(len(final_df))
        
    # the final_df is filtered to include only new rows. 
    new_rows = final_df.iloc[new_rows,:]
    
    return new_rows

# gets new rows. 
new_rows = get_new_rows(folder_path, final_df)
new_rows.reset_index(drop = True, inplace = True)
new_rows.head(4)

# gets a dataframe. 
def get_weekly_avg(data):
    
    weekly_avgs = []
    for row in range(0,len(data)):
        scores = []
        for activity in range(4,len(data.columns)):
            scores.append(data.iloc[row,activity])
        
        weekly_avg = get_avg(scores)
        weekly_avgs.append(weekly_avg)
        
    return weekly_avgs

# inserts the total column. 
final_df.insert(loc = 12, column = 'Total', value = get_weekly_avg(final_df))

# filters non-scoring months (not yet completed). 
final_df = final_df.loc[final_df.Total>0,:]
final_df.head(4)

,Name (ar),Condition (ar),Week,Activity 1,Activity 2,Activity 3,Activity 4,Activity 5,Activity 6,Activity 7,Activity 8,Activity 9,Total
0,احمد الحسين,شلل دماغي,1,1.666667,1.888889,0.315789,NaN,NaN,NaN,NaN,NaN,NaN,1.102339
24,اسلام حمادة,شلل دماغي,1,2.000000,1.333333,0.769231,NaN,NaN,NaN,NaN,NaN,NaN,1.051282
48,اسيا عبد الله,شلل دماعي,1,1.444444,1.888889,0.315789,NaN,NaN,NaN,NaN,NaN,NaN,1.102339


### 2.3 Translate new rows

This cell translates new rows with the Google translation API. If the Google rate limit is reached, the Bing translation API is used. If this still does not work, the rows will be labelled "not translated" and later rejected from being added the the old dataframe.

In [7]:
def get_translations(folder_path, final_df):
    
    # translates the names from arabic to english with google translate. 
    # if google doesn't work, it uses bing. 
    # If bing doesn't work, it says 'Not translated'. 
    english_name = []
    english_condition = []
    for i  in range(len(new_rows)):
        try:
            name = ts.google(new_rows.loc[:,'Name (ar)'][i], from_language='ar', to_language='en')
            condition = ts.google(new_rows.loc[:,'Condition (ar)'][i], from_language='ar', to_language='en')
        except:
            try:
                name = ts.bing(new_rows.loc[:,'Name (ar)'][i], from_language='ar', to_language='en')
                condition = ts.bing(new_rows.loc[:,'Condition (ar)'][i], from_language='ar', to_language='en')
            except:
                name = 'Not translated'
                condition = 'Not translated'
                
        english_name.append(name.title())
        english_condition.append(condition.title())
        
    d = {'Name': english_name,
         'Condition': english_condition}
        
    translation_df = pd.DataFrame(d)
            
    return translation_df

# gets the transaltions.
translations_df = get_translations(folder_path, final_df)
translations_df.head(4)

,Name,Condition
0,Ahmed Al -Hussein,Brain Paralysis
1,Ahmed Al -Hussein,Brain Paralysis
2,Ahmed Al -Hussein,Brain Paralysis
3,Ahmed Al -Hussein,Brain Paralysis


### 2.4 Update dataframe

This cell integrates the new rows whose transaltions were succesfully completed with the old dataframe, allowing the process to be done iteratively as the team finds additional documents which need to be transalted and added.

In [8]:
def get_updated_df(folder_path, final_df):

    # resets the index for data frames. 
    old_df.reset_index(drop=True, inplace=True)
    new_rows.reset_index(drop=True, inplace=True)

    # creates a new column for the English translation of names. 
    if len(new_rows) > 0:
        new_rows.insert(loc = 2, column = 'Name (en)', value = translations_df.Name)
        new_rows.insert(loc = 3, column = 'Condition (en)', value = translations_df.Condition)
        
        # combined the old data frame and new rows.
        updated_df = pd.concat([old_df, new_rows]).reset_index(drop=True)
        
    else:
        updated_df = old_df
        
    # returns a number for those rows that were succesfully translated. 
    translated_rows = []
    non_translated_rows = 0
    for i in range(len(updated_df)):
        if updated_df.loc[i,'Name (en)'] != 'Not Translated' and updated_df.loc[i,'Condition (en)'] != 'Not Translated':
            translated_rows.append(i)
        else:
            non_translated_rows = non_translated_rows +1
    
    # prints a warning message detailing how many rows were not translated. 
    if non_translated_rows > 0:
        message = str(non_translated_rows) + ' rows were not translated. Please try again in a couple of hours.'
        warnings.warn(message)
            
    # filters non-translated rows from the updated data frame. 
    updated_df = updated_df.iloc[translated_rows,:]
        
    return updated_df
    
# gets the updated data frame. 
updated_df = get_updated_df(folder_path, final_df).sort_values('Name (en)').reset_index(drop=True)
updated_df.head(4)

,Name (ar),Condition (ar),Name (en),Condition (en),Week,Activity 1,Activity 2,Activity 3,Activity 4,Activity 5,Activity 6,Activity 7,Activity 8,Activity 9,Total
0,عبد الرحمن حمامة,شلل دماغي,Abdul Rahman Hamama,Brain Paralysis,1,3.0,3.0,3.0,2.0,2.0,1.0,NaN,NaN,NaN,2.0
1,احمد الحسين,شلل دماغي,Ahmed Al -Hussein,Brain Paralysis,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,احمد الحسين,شلل دماغي,Ahmed Al -Hussein,Brain Paralysis,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,احمد الحسين,شلل دماغي,Ahmed Al -Hussein,Brain Paralysis,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Save data frame

This cell saves the updated dataframe with the additional rows as a csv file in the same folder under the same name, allowing the process to be repeated in the future should additional documents emerge.

In [9]:
# saves the updated spreadsheet. 
path = folder_path + 'Physiotherapy Database.csv'
updated_df.to_csv(path, index = False)